<h1> Tutorial 3.1 - The Min Oscillator </h1>

The Min oscillator is a well characterized spatial phenomena that occurs in *E. coli*. The system of reactions set up an oscillating wave that travels from end to end in the *E. coli* cell. Here we will examine how to set up a simulation of the Min system using pyLM.<br/>

You can read about the Min system [here](https://en.wikipedia.org/wiki/Min_System).

In [ ]:
import os
from pyLM import *
from pyLM.units import *
from pySTDLM.PostProcessing import *
import math as m
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Create our simulation object
sim=RDME.RDMESimulation(dimensions=micron(1.024,1.024,4.096), spacing=nm(32), name="Min Oscillator")

<h2> Build a capsid cell </h2>
This is a built in function that builds a spherocylinder (pill shaped) cell in the center of the domain with the specified length (total end to end) and the diameter.  The membrane thickness is also specified.  This may need to be increased to about 2x the lattice spacing, otherwise holes can form in the membrane.  The built in functions [```buildCapsidCell()```](http://www.scs.illinois.edu/schulten/lm/documentation/classpy_l_m_1_1_r_d_m_e_1_1_r_d_m_e_simulation.html#a07a10a44bacbb0f781115425c0124d59) and [```buildSphericalCell()```](http://www.scs.illinois.edu/schulten/lm/documentation/classpy_l_m_1_1_r_d_m_e_1_1_r_d_m_e_simulation.html#a855e223046766bcaa78d9f6f0c31c9f8) create 2 additional regions named "membrane" and "cytoplasm" which represent  what their name implies.  The extracellular space is the default region type.

In [ ]:
# Build a capsid cell
_ = sim.buildCapsidCell(length=micron(4), diameter=micron(1), membraneThickness=nm(64))

<h2> Species </h2>

minDadp - MinD protein with an ADP molecule bound <br/>
minDatp - MinD protein with an ATP molecule bound <br/>
minDm - Membrane bound MinD Protein<br/>
minE - MinE protein <br/>
minDEm - Membrane bound MinD protein also bound to a MinE protein <br/>

In [ ]:
# define our chemical species
_ = sim.defineSpecies(['minDadp', 'minDatp', 'minDm', 'minE', 'minDEm'])

<h2> Setting up the Simulation Regions </h2>

We modify cytoplasm by specifying the diffusion rate in cytoplasm regions. We also specify the reactions that occur in the cytoplasm.

In [ ]:
# Modify the cytoplasm to add diffusion rates and reactions
_ = sim.modifyRegion('cytoplasm').setDefaultDiffusionRate(2.5e-12).addReaction(reactant='minDadp', product='minDatp', rate=0.5)

Here we modify the membrane to add diffusion properties and reactions. Here the diffusion rate of 'minDm' and 'minDEm' on the membrane is set specifically, demonstrating how diffusion coefficients can be customized for each region. 

In [ ]:
# Modify the membrane to add reactions
V=3.14*4.0e-6*0.5e-6*0.5e-6*1000.0 # Liters
N_A=6.022e23                # molecules/mole
scalar=1.0/(N_A*V)

sim.modifyRegion('membrane') \
    .setDefaultDiffusionRate(2.5e-12) \
    .setDiffusionRate(species='minDm',  rate=1e-14) \
    .setDiffusionRate(species='minDEm', rate=1e-14) \
    .addReaction( 'minDatp', 'minDm',   rate=0.78) \
    .addReaction(('minDatp','minDm'), ('minDm','minDm'),rate=9e6*scalar) \
    .addReaction(('minDm','minE'),     'minDEm',     rate=5.56e7*scalar) \
    .addReaction( 'minDEm',           ('minE','minDadp'),rate=0.7)

<h2> Set diffusive properties between regions </h2>
You can set the transition rate between two regions by calling [```setTransitionRate()```](http://www.scs.illinois.edu/schulten/lm/documentation/classpy_l_m_1_1_r_d_m_e_1_1_r_d_m_e_simulation.html#a3350eaccd67aa7c7bd27d929e9431406) on the simulation object, specifying the species and the source and destination regions. This is a one way transition rate, and if the particle is to be able to move both ways the reverse transition rate must be specified with a similar call with the "via" and "to" arguments reversed.  Alternatively, a shortcut function [```setTwoWayTransitionRate()```](http://www.scs.illinois.edu/schulten/lm/documentation/classpy_l_m_1_1_r_d_m_e_1_1_r_d_m_e_simulation.html#a6450285793604051601cd4ad07bf9052) will set up diffusion both directions


In [ ]:
# Set diffusive properties between regions
sim.setTransitionRate(species='minDatp', via='cytoplasm', to='membrane', rate=2.5e-12)
sim.setTransitionRate(species='minDadp', via='cytoplasm', to='membrane', rate=2.5e-12)
sim.setTransitionRate(species='minE', via='cytoplasm', to='membrane', rate=2.5e-12)
sim.setTransitionRate(species='minDatp', to='cytoplasm', via='membrane', rate=2.5e-12)
sim.setTransitionRate(species='minDadp', to='cytoplasm', via='membrane', rate=2.5e-12)
_ = sim.setTransitionRate(species='minE', to='cytoplasm', via='membrane', rate=2.5e-12)


In [ ]:
# Populate the model with particles
sim.addParticles(species='minDatp', region='cytoplasm', count=1758)
sim.addParticles(species='minDadp', region='cytoplasm', count=1758)
_ = sim.addParticles(species='minE',    region='cytoplasm', count=914)

In [ ]:
# Set simulation Parameters
sim.setTimestep(microsecond(200))
sim.setWriteInterval(1.0)
sim.setLatticeWriteInterval(1.0)
sim.setSimulationTime(150)

In [ ]:
# Save simulation
filename="T3.1-MinSystem.lm"
try:
    os.remove(filename)
except:
    pass
sim.save(filename)

In [ ]:
# Interrogate Simulation
sim

<h2> Running the Simulation </h2>

Running this simulation should take about 15-20 minutes. We have prerun this simulation os that you don't haave to wait for it to complete.

In [ ]:
# Run Simulation
#sim.run(filename, method='lm::rdme::MpdRdmeSolver', replicates=1)

<h2> Post-Processing </h2>

Here we plot the waves that arise by plotting the kymograph (a time versus position plot).  These commands take a few minutes to extract the information from the simulation output.

In [ ]:
# Plot Kymogrpah for MinD
filename='T3.1-MinSystem-prerun.lm'
plotOccupancyKymograph(filename, species='minDm')
plt.show()

<h2> Assignment 4 </h2>

1. How long does it take for the oscillatory waves to emerge?

2. Estimate the oscillation frequency from the Kymographs. Try re-running the simulation  packing the cell full of obstacles.  Do you get a different oscillation frequency?

3. (Challenge) Try rerunning the simulation with a spherical cell.  To do this use the [```buildSphericalCell()```](http://www.scs.illinois.edu/schulten/lm/documentation/classpy_l_m_1_1_r_d_m_e_1_1_r_d_m_e_simulation.html#a855e223046766bcaa78d9f6f0c31c9f8) function. Also, reduce the simulation box size (try 1$\times$1$\times$1 $\mu$m) to increase the time to solution.  Can you identify why stochasticity is important in the Min System?